In [16]:
import requests
import json
import pandas as pd
import numpy as np
df = pd.DataFrame()
def tokmb (num):
    convNum = ''
    if (num >= 1000 and num < 1000000):
        num = round (num/1000.00,2)
        round (num, 2)
        convNum = str(num) + " K"
        return convNum
    elif (num >= 1000000 and num < 1000000000):
        num = round (num/1000000.00,2)
        round (num, 2)
        convNum = str(num) + " M"
        return convNum
    elif (num >= 1000000000 and num < 1000000000000):
        num = round (num/1000000000.00,2)
        convNum = str(num) + " B"
        return convNum
def get_avg_sector_rates():
    sub = ''
    cols_to_use = [0,1,2,3,4,5,6,7]
    csv_file_path = r"../../assets/companylist.csv"
    df = pd.read_csv(csv_file_path, 
        usecols= cols_to_use,
        encoding='utf-8'
        )
    df['IPOyear'] = df['IPOyear'].astype(str)
    df['IPOyear'] = df['IPOyear'].str.strip('.0')
    df = df.rename(index=str, columns = {"Symbol": "SYM", "Name": "CO.", "LastSale": "LP", "MarketCap": "MRKCAP", "IPOyear": "YR", "Sector": "SEC", "industry": "IND", "Summary Quote": "SUMQTE"})
    df['SYM'] = df['SYM'].str.strip()
    df['SEC'] = df['SEC'].str.strip()
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Non-Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=='Consumer Services', 'Consumer Industry', df['SEC'])
    df["SEC"] = df["SEC"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Non-Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Technology", "Technology/Energy")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Energy", "Technology/Energy")
    df = df.groupby('SEC_CONV').head(100)
    df = df.iloc[:, [8,3]]
    df['MRKCAP'] = df['MRKCAP'].str.replace('$', '')
    df['MRKCAP'] = df['MRKCAP'].str.replace(',', '')
    df['MRKCAP'] = df['MRKCAP'].str.replace('M', ' M')
    df['MRKCAP'] = df['MRKCAP'].str.replace('B', ' B')
    df['SEC_CONV'] = df['SEC_CONV'].fillna(value='No Industry Identified')
    df['MRKCAP'] = df['MRKCAP'].fillna(value='0 D')
    df.dropna(inplace = True)
    df[['MRKCAP_AMT','MRKCAP_TYPE']] = df.MRKCAP.str.split(" ",expand=True,)
    millions = df['MRKCAP_AMT'].str.replace('.', '').fillna(value='0').astype(int).multiply(other = 100, fill_value = 5)
    billions = df['MRKCAP_AMT'].str.replace('.', '').fillna(value='0').astype(int).multiply(other = 1000000, fill_value = 5)
    df.loc[df['MRKCAP_TYPE'] == 'M', 'Conversion'] = millions
    df.loc[df['MRKCAP_TYPE'] == 'B', 'Conversion'] = billions
    df.loc[df['MRKCAP_TYPE'] == 'D', 'Conversion'] = 0
    df.loc[df['MRKCAP_TYPE'] == 'None', 'Conversion'] = df['MRKCAP_AMT'].fillna(value='0')
    df['Conversion'] = df['Conversion'].fillna(value='0')
    df['Conversion'] = df['Conversion'].astype(int)
    res = df.groupby(['SEC_CONV'])['Conversion'].mean().reset_index()
    for x in range (0, len(res['Conversion'])):
        res ['Conversion'][x] = tokmb(res ['Conversion'][x])
    res = res.to_json(orient='records')
#     res = json.dumps(res)
    #res = res.replace('\\','')
    return res
get_avg_sector_rates()

c:\users\ksummersill\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'[{"SEC_CONV":"Basic Industries","Conversion":"75.11 M"},{"SEC_CONV":"Capital Goods","Conversion":"91.96 M"},{"SEC_CONV":"Consumer Industry","Conversion":"80.41 M"},{"SEC_CONV":"Finance","Conversion":"56.36 M"},{"SEC_CONV":"Health Care","Conversion":"61.03 M"},{"SEC_CONV":"Miscellaneous","Conversion":"61.35 M"},{"SEC_CONV":"No Industry Identified","Conversion":"31.63 M"},{"SEC_CONV":"Public Utilities","Conversion":"27.97 M"},{"SEC_CONV":"Technology\\/Energy","Conversion":"72.44 M"},{"SEC_CONV":"Transportation","Conversion":"258.05 M"}]'